# Imports

In [60]:
import numpy as np
import pandas as pd
import random as rand
import matplotlib.pyplot as plt
import tensorflow as tf

from keras.utils import to_categorical
from keras.layers import Dense, Activation, Conv2D,Conv2DTranspose, Dropout, Reshape, MaxPooling2D, Flatten
from keras.models import Sequential, load_model
from keras.losses import BinaryCrossentropy
from keras.optimizers import Adam


from sklearn.model_selection import train_test_split

# Data importing and pre-processing

In [51]:
df = pd.read_csv('dataset.csv', header=None)


df = df.values.reshape(60, 64, 64, 1)

labels = np.zeros(60)

x_real_train, x_real_test = train_test_split(df, test_size=0.2) #12 test values
y_real_train, y_real_test = train_test_split(labels, test_size=0.2)

# Generator Model

Create points in latent space to be fed into generator

In [104]:
def generate_generator_input():
    input = np.random.normal(3,2.5,size=(1,100))
    
    return input

In [106]:
def make_generator():
    model = Sequential()
    
    model.add(Dense(60*8*8, input_shape=(100,)))
    model.add(Activation("relu"))
    model.add(Reshape((8,8,60)))
    
    model.add(Conv2DTranspose(1, (1,1), strides=(2,2), padding='same', input_shape=(8,8,60)))
    model.add(Activation("relu"))
  
    
    model.add(Conv2DTranspose(1, (1,1), strides=(2,2), padding='same', input_shape=(16,16,60)))
    model.add(Activation("relu"))

    model.add(Conv2DTranspose(1, (1,1), strides=(2,2), padding='same', input_shape=(32,32,60)))
    model.add(Activation("tanh"))
    
              
    model.summary()
    
    return model
    
    

# Discriminator Model

In [107]:

def make_discriminator():
    
    # 1st set of layers
    model = Sequential()
    model.add(Conv2D(128, kernel_size=5, padding="same", input_shape=(64,64,1)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
  
    
    # output layer
    model.add(Flatten())
    model.add(Dense(1)) # Binary classification (2 outputs), so only 1 dense layer needed
    
    model.summary()
    return model
    

# Training

First, create the models from the functions

In [108]:
gen_model = make_generator()

noise = generate_generator_input()
generated_map = gen_model(noise, training = False)



Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_41 (Dense)            (None, 3840)              387840    
                                                                 
 activation_96 (Activation)  (None, 3840)              0         
                                                                 
 reshape_18 (Reshape)        (None, 8, 8, 60)          0         
                                                                 
 conv2d_transpose_55 (Conv2  (None, 16, 16, 1)         61        
 DTranspose)                                                     
                                                                 
 activation_97 (Activation)  (None, 16, 16, 1)         0         
                                                                 
 conv2d_transpose_56 (Conv2  (None, 32, 32, 1)         2         
 DTranspose)                                         

Next, test the untrained discriminator on the map of noise generated before
Negative values means fake, positive means real

In [109]:
disc_model = make_discriminator()
decision = disc_model(generated_map)
print(decision)

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 64, 64, 128)       3328      
                                                                 
 activation_100 (Activation  (None, 64, 64, 128)       0         
 )                                                               
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 32, 32, 128)       0         
 g2D)                                                            
                                                                 
 flatten_7 (Flatten)         (None, 131072)            0         
                                                                 
 dense_42 (Dense)            (None, 1)                 131073    
                                                                 
Total params: 134401 (525.00 KB)
Trainable params: 13

## Loss and Optimizers

In [110]:
cross_entropy = BinaryCrossentropy(from_logits=True)

Discriminator loss, adapted from: https://www.tensorflow.org/tutorials/generative/dcgan

In [113]:
def discrim_loss(real_output, fake_output):
    real_loss = cross_entropy(np.ones_like(real_output), real_output)
    fake_loss = cross_entropy(np.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

Generator loss

In [114]:
def generator_loss(fake_output):
    return cross_entropy(np.ones_like(fake_output), fake_output)

Optimizers:

In [115]:
gen_optimizer = Adam(1e-4)
disc_optimizer = Adam(1e-4)

## Training

training parameters:

In [116]:
N_EPOCHS = 48

VERBOSE = 1


Training step

In [120]:
@tf.function
def training_step(current_batch):
    
    noise_sample = generate_generator_input()

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

        generated_map = gen_model(noise_sample, training=True)

        real_output = disc_model(current_batch, training=True)
        fake_output = disc_model(generated_map, training = True)

        gen_loss = generator_loss(fake_output=fake_output)
        disc_loss = discrim_loss(real_output=real_output, fake_output=fake_output)

    gen_gradients = gen_tape.gradient(gen_loss, gen_model.trainable_variables)
    disc_gradients = disc_tape.gradient(disc_loss, disc_model.trainable_variables)

    gen_optimizer.apply_gradients(zip(gen_gradients, gen_model.trainable_variables))
    disc_optimizer.apply_gradients(zip(disc_gradients, disc_model.trainable_variables))

Train models

In [121]:
def train(dataset, N_EPOCHS):
    for epoch in range(N_EPOCHS):

        for map_batch in dataset:
            print(map_batch.shape)
            training_step(map_batch)
    
    input_for_map_after_training = generate_generator_input()
    map_after_training = gen_model(input_for_map_after_training, training=False)

    print(map_after_training)



Train GAN

In [122]:
print(x_real_train.shape)

train(x_real_train, N_EPOCHS)


(48, 64, 64, 1)
(64, 64, 1)


TypeError: in user code:

    File "/tmp/ipykernel_102226/2522044098.py", line 10, in training_step  *
        real_output = disc_model(current_batch, training=True, verbose=VERBOSE)
    File "/home/ubuntu/.local/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None

    TypeError: Sequential.call() got an unexpected keyword argument 'verbose'
